# Soft-Min Torch Computator Walkthrough

This notebook explains how the differentiable `SoftMinTorchComputator` integrates with `BPEngine`, mirroring the example script while adding diagnostic comparisons against the classic Min-Sum variant.


## What You Will Learn

- Instantiate the same cycle graph used in the example script with deterministic cost tables.
- Run the PropFlow `BPEngine` twice: once with the soft-min torch computator and once with Min-Sum.
- Inspect variable assignments, global costs, and individual messages to see how soft-min smooths hard minimum operations.


In [ ]:
# Import numpy for deterministic random number handling.
import numpy as np
# Import torch to exercise the differentiable computator (raises ImportError if missing).
import torch
# Import the PropFlow engine and factor graph builder to mirror the example script.
from propflow import BPEngine, FGBuilder
# Import the classic Min-Sum computator for baseline comparisons.
from propflow.bp.computators import MinSumComputator
# Import the SoftMinTorchComputator showcased in the new example test.
from propflow.nn.torch_computators import SoftMinTorchComputator
# Quick sanity check to confirm torch exposes the Tensor class before continuing.
assert hasattr(torch, "Tensor"), "PyTorch must be installed to run this notebook."
# Seed numpy so generated cost tables remain reproducible across runs.
np.random.seed(7)
# Seed torch so any tensor-based randomness is repeatable as well.
torch.manual_seed(7)


## Build a Deterministic Cycle Graph

We reproduce the five-variable cycle used in `examples/torch_softmin_demo.py`, but we seed the cost tables so every run stays identical.


In [ ]:
# Define a helper for building the small cycle graph demonstrated below.
def build_demo_cycle_graph(seed: int, num_vars: int = 5, domain: int = 3):
    # Initialize a dedicated RNG so generated cost tables stay reproducible.
    rng = np.random.default_rng(seed)
    # Define a local factory that mirrors the integer cost table generator from the example script.
    def seeded_cost_table(n: int, domain_size: int, low: int = 1, high: int = 20):
        # Draw integer-valued costs and expose them as floating point arrays for the computators.
        return rng.integers(low=low, high=high, size=(domain_size,) * n).astype(float)
    # Build the cycle graph via FGBuilder so structure matches the example topology.
    graph = FGBuilder.build_cycle_graph(
        num_vars=num_vars,
        domain_size=domain,
        ct_factory=seeded_cost_table,
        ct_params={"low": 1, "high": 20},
    )
    # Return the freshly built factor graph so callers get an independent copy.
    return graph


## Run Both Engines

We now create two independent graphs and run the BP engine with the soft-min and Min-Sum computators respectively.


In [ ]:
# Create a fresh factor graph for the differentiable engine so that message state starts cleanly.
soft_graph = build_demo_cycle_graph(seed=21)
# Mirror the same graph for the Min-Sum baseline to compare outputs fairly.
minsum_graph = build_demo_cycle_graph(seed=21)
# Instantiate the SoftMinTorchComputator with a low temperature to approximate hard min-sum behavior.
soft_computator = SoftMinTorchComputator(tau=1e-2, device="cpu")
# Instantiate the classic Min-Sum computator as the reference implementation.
minsum_computator = MinSumComputator()
# Create the Soft-Min BP engine and enable per-step history captures for analysis.
soft_engine = BPEngine(factor_graph=soft_graph, computator=soft_computator, use_bct_history=True)
# Create the Min-Sum BP engine with the same history configuration for apples-to-apples comparisons.
minsum_engine = BPEngine(factor_graph=minsum_graph, computator=minsum_computator, use_bct_history=True)
# Run the soft-min engine for a modest number of iterations without persisting history artifacts.
soft_engine.run(max_iter=10, save_json=False, save_csv=False)
# Run the Min-Sum engine with identical runtime settings.
minsum_engine.run(max_iter=10, save_json=False, save_csv=False)


## Compare Assignments and Global Cost

With both engines executed, we verify they converge to the same assignments and overall objective value.


In [ ]:
# Capture the final assignments produced by the soft-min engine.
soft_assignments = soft_engine.assignments
# Capture the final assignments produced by the Min-Sum baseline.
minsum_assignments = minsum_engine.assignments
# Pull the final global cost from the soft-min engine for comparison.
soft_cost = soft_engine.graph.global_cost
# Pull the final global cost from the Min-Sum engine as the reference.
minsum_cost = minsum_engine.graph.global_cost
# Display the assignments so we can verify they match exactly.
print("Soft-min assignments:", soft_assignments)
# Display the baseline assignments for side-by-side inspection.
print("Min-Sum assignments:", minsum_assignments)
# Display the global costs to confirm the objectives coincide.
print("Soft-min global cost:", soft_cost)
# Display the baseline cost for direct comparison.
print("Min-Sum global cost:", minsum_cost)


## Inspect Individual Messages

The soft-min computator smooths the hard minimum. By exploring the recorded messages from the first BP step, we can see how closely the softened values match Min-Sum.


In [ ]:
# Access the recorded per-step messages from the soft-min engine history.
soft_step_messages = soft_engine.history.step_messages
# Select the first step to inspect the earliest factor-to-variable communication.
soft_first_step = soft_step_messages.get(0, [])
# Filter the messages to those originating from factor nodes so we focus on R messages.
soft_factor_messages = [msg for msg in soft_first_step if msg.sender.startswith("f")]
# Ensure we have at least one factor message to inspect.
assert soft_factor_messages, "Expected at least one factor message in step 0."
# Pick the first factor message for detailed inspection.
soft_sample_message = soft_factor_messages[0]
# Repeat the same logging extraction for the Min-Sum baseline.
minsum_step_messages = minsum_engine.history.step_messages
# Select the same chronological step for the baseline engine.
minsum_first_step = minsum_step_messages.get(0, [])
# Build a lookup keyed by (sender, recipient) so we can match the same edge.
minsum_lookup = {
    (msg.sender, msg.recipient): msg
    for msg in minsum_first_step
}
# Fetch the corresponding baseline message using the same sender and recipient identifiers.
minsum_sample_message = minsum_lookup[(soft_sample_message.sender, soft_sample_message.recipient)]
# Convert the stored message payload into a numpy array for the soft-min engine.
soft_vector = np.array(soft_sample_message.data)
# Convert the baseline payload into a numpy array as well.
minsum_vector = np.array(minsum_sample_message.data)
# Compute the element-wise difference to illustrate the soft-min smoothing behavior.
difference_vector = soft_vector - minsum_vector
# Print the soft-min message values for inspection.
print("Soft-min R message:", soft_vector)
# Print the Min-Sum message values for side-by-side comparison.
print("Min-Sum R message:", minsum_vector)
# Print the difference so we can quantify how closely the soft approximation tracks the hard minimum.
print("Difference (soft - min):", difference_vector)


## Takeaways

- The soft-min torch computator matches Min-Sum assignments and global cost while remaining differentiable.
- Message traces reveal the soft-min outputs shadow the hard min values with minor smoothing that depends on the temperature `tau`.
- You can lower `tau` for closer agreement or raise it when smoother gradients are required for learning workflows.
